<a href="https://colab.research.google.com/github/fatday/My-Grad-Math-Works/blob/main/NumericalMethod/a6q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# Finding Exact Solution

For the problem (9.19) in Larsson-Thomee, we can get the Fourier Transform of the exact solution

$$
\hat{\mu}(\xi,t)=e^{-\xi^2 t}\hat{v}(\xi)
$$

By the (8.3) in Larsson-Thomee, we have

\begin{align*}
u(x,t)&=\frac{1}{\sqrt{4\pi t}}\int_{\mathbb{R}} v(y) e^{-(x-y)^2/4}dy\\
&=\frac{1}{\sqrt{4\pi t}}\int_{\mathbb{R}} \sin(\pi y) e^{-(x-y)^2/4}dy+\frac{1}{\sqrt{4\pi t}}\int_{\mathbb{R}} \sin(3 \pi y) e^{-(x-y)^2/4}dy
\end{align*}

Now we will calculate these integrals, since $\sin(\pi y)=\frac{1}{2i}[e^{i\pi y}-e^{-i\pi y}]$, then by Gaussian integral we have

\begin{align*}
\frac{1}{\sqrt{4\pi t}}\int_{\mathbb{R}} \sin(\pi y) e^{-(x-y)^2/4}dy&=
\frac{1}{2i}\cdot \frac{1}{\sqrt{4\pi t}}\int_{\mathbb{R}} e^{i\pi y}e^{-(x-y)^2/4}dy-\frac{1}{2i}\cdot \frac{1}{\sqrt{4\pi t}}\int_{\mathbb{R}} e^{-i\pi y}e^{-(x-y)^2/4}dy\\
&=\frac{1}{2i}\cdot \frac{1}{\sqrt{4\pi t}}\cdot \sqrt{4\pi t}e^{-\pi^2 t} e^{i\pi x}-\frac{1}{2i}\cdot \frac{1}{\sqrt{4\pi t}}\cdot \sqrt{4\pi t}e^{-\pi^2 t} e^{-i\pi x}\\
&=\frac{e^{-\pi^2 t}}{2i}[e^{i\pi y}-e^{-i\pi y}]\\
&=\sin(\pi x)e^{-\pi^2 t}
\end{align*}

Similarly, we can get

$$
\frac{1}{\sqrt{4\pi t}}\int_{\mathbb{R}} \sin(\pi y) e^{-(x-y)^2/4}dy=\sin(3\pi x)e^{-9\pi^2 t}
$$

This gives the exact solution:

$$
u(x,t)=\sin(\pi x)e^{-\pi^2 t}-\sin(3\pi x)e^{-9\pi^2 t}
$$

In [2]:
def v(x):
  return np.sin(np.pi*x) - np.sin(3*np.pi*x)

In [3]:
def u(x,t):
    return np.sin(np.pi*x)*np.exp(-np.pi**2*t)-np.sin(3*np.pi*x)*np.exp(-9*np.pi**2*t)

In [153]:
def forward_Euler_U(h, k, x, t):
    # constant / size setup
    X = np.arange(0,1+h,h)
    lambda_val = k/h**2
    N = int(t/k + 1)
    target_x_index = int(x/h)

    # init val & matrix
    U0 = np.zeros(len(X))
    U0[1:-1] = v(X[1:-1])
    n = len(X)
    A = ((np.diag((1-2*lambda_val) * np.ones(n), 0)  + np.diag(lambda_val * np.ones(n-1), 1) + np.diag(lambda_val * np.ones(n-1), -1)))

    # iterate
    for i in range(1,N):
        U0 = A @ U0
        U0[0], U0[-1] = 0, 0

    return U0[target_x_index]

In [154]:
def Crank_Nicolson_U(h, k, x, t):
    # constant / size setup
    X = np.arange(0,1+h,h)
    lambda_val = k/(h**2)
    N = int(t/k + 1)
    target_x_index = int(x/h)

    # init val & matrix
    U0 = np.zeros(len(X))
    U0[1:-1] = v(X[1:-1])
    n = len(X)

    # A, B matrix
    A = ((np.diag((1-lambda_val) * np.ones(n), 0)  + np.diag(0.5*lambda_val * np.ones(n-1), 1) + np.diag(0.5*lambda_val * np.ones(n-1), -1)))
    B = ((np.diag((1+lambda_val) * np.ones(n), 0)  + np.diag(-0.5*lambda_val * np.ones(n-1), 1) + np.diag(-0.5*lambda_val * np.ones(n-1), -1)))

    for i in range(1, N):
        rightU = A @ U0
        U0 = np.linalg.solve(B, rightU)
        U0[0], U0[-1] = 0, 0
    return U0[target_x_index]

In [155]:
h = 1/10

k0 = 1/10
k1 = 1/600
k2 = 1/300
k3 = 1/100

x = 1/2
t = 1

In [156]:
u(x,t)

5.172318620381234e-05

# Euler method - h = 1/10, k = 1/600 Error:

In [164]:
round(abs(forward_Euler_U(h, k1, x, t) - u(x,t)), 15)

9.290894e-09

# Euler method - h = 1/10, k = 1/300 Error:

In [165]:
round(abs(forward_Euler_U(h, k2, x, t) - u(x,t)), 15)

4.08879348e-06

# Euler method - h = 1/10, k = 1/100 Error:

In [166]:
round(abs(forward_Euler_U(h, k3, x, t) - u(x,t)), 9) # not stable

2.551994391139398e+30

# Crank-Nicolson method - h = k = 1/10

In [167]:
round(abs(Crank_Nicolson_U(h, k0, x, t) - u(x,t)), 9)

0.007050355